In [69]:
import pandas as pd
import numpy as np
import matplotlib as plt
import sys
sys.path.append('../Task 1/')
from efficient_apriori import apriori
from improved_apriori import Improved_Apriori
import json
import time
import os
import itertools
import ast
from tqdm import tqdm
import collections
import math
import random

In [70]:
# Process the dataset by chunks into tid and the list of bakerys the user watched
def process_bakery_chunk(df, carry_over):
    if carry_over is not None:
        df = pd.concat([carry_over, df])
    groups = df.groupby('tid')['article'].apply(list)
    last_user = df.iloc[-1]['tid']
    if last_user in groups:
        carry_over = df[df['tid'] == last_user]
        groups = groups.drop(last_user)
    else:
        carry_over = None
    return groups, carry_over

In [71]:
carry_over = None
chunksize = 100000 # adjust this value depending on your available memory
# Might have to figure out a way to shuffle the dataset 
if(not os.path.exists('processed_bakery_output3.txt')):
    with open('processed_bakery_output3.txt', 'w') as f:
        for chunk in pd.read_csv('Bakery.csv', chunksize=chunksize):
            groups, carry_over = process_bakery_chunk(chunk, carry_over)
            for user, bakery_list in groups.items():
                f.write(f'{user} {bakery_list}\n')

        # don't forget to process the last carry_over
        if carry_over is not None:
            groups, _ = process_bakery_chunk(carry_over, None)
            for user, bakery_list in groups.items():
                f.write(f'{user} {bakery_list}\n')

In [72]:
# Shuffle the text in chunks 
def shuffle_large_file(file_name, output_file_name, chunk_size):
    with open(file_name, 'r') as f:
        while True:
            lines = list(itertools.islice(f, chunk_size))
            if not lines:
                break
            random.shuffle(lines)
            with open(output_file_name, 'a') as out:
                out.write(''.join(lines))


# Call the function with your parameters
if(not os.path.exists('processed_bakery_output_shuffled3.txt')):
    shuffle_large_file('processed_bakery_output3.txt', 'processed_bakery_output_shuffled3.txt', 3000000)

In [73]:
def read_file_in_partitions(file_path, partition_size):
    with open(file_path, 'r') as file:
        partition = []
        for line in file:
            partition.append(line)
            if len(partition) >= partition_size:
                yield partition
                partition = []
        if partition:  # yield any remaining lines
            yield partition

In [74]:
# Global variable to get the counts of all itemsets
global_counts = {}
def generate_global_counts(partition, global_candidates):

    # For 1th itemset, generate the transaction id list for the ith partition 
    transaction_id_dict = collections.defaultdict(list)
    for transaction_id in partition:
        for item in partition[transaction_id]:
            item_tuple = (item,)
            transaction_id_dict[item_tuple].append(transaction_id)

    # Filter based on the global candidates formed
    transaction_ids_dict = {item: transaction_ids for item, transaction_ids in transaction_id_dict.items() if item in global_candidates[1]}

    # Get the global count of all 1th itemset
    for item in transaction_id_dict:
        if(len(item) not in global_counts):
            global_counts[len(item)] = {}
        if(item not in global_counts[len(item)]):
            global_counts[len(item)][item] = len(transaction_id_dict[item])
        else:
            global_counts[len(item)][item] += len(transaction_id_dict[item])

    # Extend to find global count of all nth itemset from the global candidates
    for i in tqdm(range(1, len(global_candidates))):
        for itemset in global_candidates[i+1]:
            transaction_ids = set(transaction_id_dict[(itemset[0],)])
            for i in range(1, len(itemset)):
                # We are only interested in the transactions where all items in itemset is present
                transaction_ids = transaction_ids.intersection(set(transaction_ids_dict.get((itemset[i],), {})))
            if(len(itemset) not in global_counts):
                global_counts[len(itemset)] = {}

            if(itemset not in global_counts[len(itemset)]):
                global_counts[len(itemset)][itemset] = len(transaction_ids)
            else:
                global_counts[len(itemset)][itemset] += len(transaction_ids)



In [75]:
file_path = 'processed_bakery_output_shuffled3.txt'
size_of_data = sum(1 for line in open(file_path))

partition_size = 10000
if partition_size < size_of_data:
    num_partitions = size_of_data // partition_size
else:
    print('Size of partition exceeds size of data')
    exit()

partition_candidates = []
global_candidates = collections.defaultdict(list)

# Step 1: Partitioning
min_support_range = np.arange(0.001, 0.007, 0.001)

for min_support in min_support_range:
    global_min_support = math.ceil((min_support * size_of_data) / num_partitions)
    start = time.time()

    # Step 2: Retrieve frequent itemset per partition
    for i, partition in enumerate(read_file_in_partitions(file_path, partition_size)):
        print(f'Partition {i + 1}:')
        dict_bakery = {}
        for line in partition:
            user, bakery_list_str = line.strip().split(' ', 1)
            bakery_list = ast.literal_eval(bakery_list_str)
            dict_bakery[user] = bakery_list
    
        improved_apriori = Improved_Apriori(dict_bakery, min_support=min_support, min_confidence=1, verbose=0)
        partition_frequent_itemset = improved_apriori.apriori()

        # Form the global candidate set from the large itemset in each partition
        for level, itemset in partition_frequent_itemset.items():
            for key in itemset.keys():
                if key not in global_candidates[level]:
                    global_candidates[level].append(key)

    min_support_count = min_support * size_of_data

    # Global counting phase
    global_counts = {}
    for i, partition in enumerate(read_file_in_partitions(file_path, partition_size)):
        print(f'Partition {i + 1}:')
        dict_bakery = {}
        for line in partition:
            user, bakery_list_str = line.strip().split(' ', 1)
            bakery_list = ast.literal_eval(bakery_list_str)
            dict_bakery[user] = bakery_list
        generate_global_counts(dict_bakery, global_candidates)

    global_frequent_itemsets = {
        level: {itemset: count for itemset, count in itemsets.items() if count >= min_support_count}
        for level, itemsets in global_counts.items()
    }

    end = time.time()

    # Count the number of frequent itemsets and find the largest size
    num_frequent_itemsets = 0
    largest_size = 0
    for level, itemsets in global_frequent_itemsets.items():
        num_frequent_itemsets += len(itemsets)
        for items, count in itemsets.items():
            largest_size = max(largest_size, len(items))

    print(f'min_support: {min_support}')
    print(f'num_partitions: {num_partitions}')
    print(f'running_time: {end - start}')
    print(f'num_frequent_itemsets: {num_frequent_itemsets}')
    print(f'largest_size_of_itemsets: {largest_size}')
    print()


Partition 1:


100%|██████████| 10/10 [00:00<00:00, 3956.89it/s]
0it [00:00, ?it/s]

Partition 2:



100%|██████████| 10/10 [00:00<00:00, 4024.09it/s]
0it [00:00, ?it/s]


Partition 3:


...

Partition 13:


100%|██████████| 3/3 [00:00<00:00, 104.70it/s]


Partition 14:


100%|██████████| 3/3 [00:00<00:00, 165.33it/s]

min_support: 0.006
num_partitions: 13
running_time: 3.1663951873779297
num_frequent_itemsets: 77
largest_size_of_itemsets: 3



In [76]:
# data = pd.read_csv('dataset/final_animedataset.csv')
# data = data[['tid', 'article']]
# grouped_data = data.groupby('tid')['article'].apply(list)
# grouped_data = grouped_data.to_dict()

In [77]:
# frequent_anime_set

In [78]:
# itemset, _ = apriori(list(grouped_data.values()), min_support = min_support, verbosity=2)

In [79]:
# itemset